### **Task 1 (3.0 points): 8x8 Tic-Tac-Toe**

In [5]:
import copy

class Problem:
    def __init__(self, current_player='X'):
        self.current_state = [[' ' for _ in range(8)] for _ in range(8)]
        self.current_player = current_player

    def terminal_test(self, state):
        return self.check_winner(state) is not None or all(all(cell != ' ' for cell in row) for row in state)

    def result(self, state, action):
        new_state = copy.deepcopy(state)
        row, col, player = action
        new_state[row][col] = player
        return new_state

    def actions(self, state, current_player):
        return [(i, j, current_player) for i in range(8) for j in range(8) if state[i][j] == ' ']

    def check_winner(self, state):
        directions = [(0, 1), (1, 0), (1, 1), (1, -1)]
        for x in range(8):
            for y in range(8):
                current_player = state[x][y]
                if current_player == ' ':
                    continue
                for dx, dy in directions:
                    nx, ny = x, y
                    count = 0
                    while 0 <= nx < 8 and 0 <= ny < 8 and state[nx][ny] == current_player:
                        nx += dx
                        ny += dy
                        count += 1
                        if count == 4:
                            return current_player
        return None

    def utility(self, state, current_player):
        opponent = 'O' if current_player == 'X' else 'X'
        utility_score = 0
        directions = [(0, 1), (1, 0), (1, 1), (1, -1)]
        for direction in directions:
            for x in range(8):
                for y in range(8):
                    count_player = 0
                    count_opponent = 0
                    for i in range(4):
                        nx, ny = x + direction[0] * i, y + direction[1] * i
                        if 0 <= nx < 8 and 0 <= ny < 8:
                            if state[nx][ny] == current_player:
                                count_player += 1
                            elif state[nx][ny] == opponent:
                                count_opponent += 1
                    if count_player == 3 and count_opponent == 0:
                        utility_score += 100
                    elif count_opponent == 3 and count_player == 0:
                        utility_score -= 1000
                    elif count_opponent == 3 and count_player == 1:
                        if 0 <= x - direction[0] < 8 and 0 <= y - direction[1] < 8:
                            if state[x - direction[0]][y - direction[1]] != current_player:
                                utility_score -= 700
                        if 0 <= x + direction[0] * 3 < 8 and 0 <= y + direction[1] * 3 < 8:
                            if state[x + direction[0] * 3][y + direction[1] * 3] != current_player:
                                utility_score -= 700
                    elif count_player == 2 and count_opponent == 0:
                        if 0 <= x + direction[0] * 3 < 8 and 0 <= y + direction[1] * 3 < 8:
                            if state[x + direction[0] * 3][y + direction[1] * 3] == opponent:
                                utility_score += 50
                            elif 0 <= x - direction[0] < 8 and 0 <= y - direction[1] < 8:
                                if state[x - direction[0]][y - direction[1]] != current_player:
                                    utility_score += 50
                    elif count_opponent == 2 and count_player == 0:
                        if 0 <= x - direction[0] < 8 and 0 <= y - direction[1] < 8:
                            if state[x - direction[0]][y - direction[1]] != current_player:
                                utility_score -= 500
        return utility_score

class SearchStrategy:
    def __init__(self, max_depth=3):
        self.max_depth = max_depth

    def alpha_beta_search(self, problem):
        current_player = problem.current_player
        opponent = 'O' if current_player == 'X' else 'X'

        def max_value(state, alpha, beta, depth):
            if depth == self.max_depth or problem.terminal_test(state):
                return problem.utility(state, current_player)
            v = float('-inf')
            for action in problem.actions(state, current_player):
                v = max(v, min_value(problem.result(state, action), alpha, beta, depth + 1))
                if v >= beta:
                    return v
                alpha = max(alpha, v)
            return v

        def min_value(state, alpha, beta, depth):
            if depth == self.max_depth or problem.terminal_test(state):
                return problem.utility(state, current_player)
            v = float('inf')
            for action in problem.actions(state, current_player):
                v = min(v, max_value(problem.result(state, action), alpha, beta, depth + 1))
                if v <= alpha:
                    return v
                beta = min(beta, v)
            return v

        for action in problem.actions(problem.current_state, current_player):
            next_state = problem.result(problem.current_state, action)
            if problem.check_winner(next_state) == current_player:
                return action

        for action in problem.actions(problem.current_state, opponent):
            next_state = problem.result(problem.current_state, action)
            if problem.check_winner(next_state) == opponent:
                return (action[0], action[1], current_player)

        alpha = float('-inf')
        beta = float('inf')
        best_value = float('-inf')
        best_action = None

        for action in problem.actions(problem.current_state, current_player):
            value = min_value(problem.result(problem.current_state, action), alpha, beta, 1)
            if value > best_value:
                best_value = value
                best_action = action
                alpha = max(alpha, value)

        return best_action

class Game:
    def __init__(self):
        self.problem = Problem()
        self.strategy = SearchStrategy()
        self.current_player = 'X'

    def switch_player(self):
        self.current_player = 'O' if self.current_player == 'X' else 'X'
        self.problem.current_player = self.current_player

    def play(self):
        first_player = input("Enter 'AI' if you want AI to go first, or 'human' if you want the human to go first: ").strip().lower()
        if first_player == 'human':
            self.current_player = 'O'
            self.problem.current_player = 'O'
        else:
            print("AI is making its first move...")
            action = self.strategy.alpha_beta_search(self.problem)
            self.problem.current_state = self.problem.result(self.problem.current_state, action)
            self.switch_player()

        while not self.problem.terminal_test(self.problem.current_state):
            self.display_board()
            if self.current_player == 'X':
                print("AI is making its move...")
                action = self.strategy.alpha_beta_search(self.problem)
                self.problem.current_state = self.problem.result(self.problem.current_state, action)
            else:
                valid_move = False
                while not valid_move:
                    try:
                        row = int(input("Enter the row (0-7): "))
                        col = int(input("Enter the column (0-7): "))
                        if self.problem.current_state[row][col] == ' ':
                            self.problem.current_state = self.problem.result(self.problem.current_state, (row, col, self.current_player))
                            valid_move = True
                        else:
                            print("Invalid move. Try again.")
                    except (ValueError, IndexError):
                        print("Invalid input. Please enter numbers between 0 and 7.")

            if self.problem.terminal_test(self.problem.current_state):
                self.display_board()
                winner = self.problem.check_winner(self.problem.current_state)
                if winner:
                    print(f"The winner is {winner}!")
                else:
                    print("The game is a draw!")
                break

            self.switch_player()

    def display_board(self):
        print("  " + "   ".join([str(i) for i in range(8)]))
        print(" +" + "---+" * 8)
        for idx, row in enumerate(self.problem.current_state):
            print(f"{idx}| " + " | ".join(row) + " |")
            print(" +" + "---+" * 8)

def main():
  game = Game()
  game.play()

### **Run Task 1: 8x8 Tic-Tac-Toe**

In [ ]:
if __name__ == "__main__":
    main()

Enter 'AI' if you want AI to go first, or 'human' if you want the human to go first: AI
AI is making its first move...
  0   1   2   3   4   5   6   7
 +---+---+---+---+---+---+---+---+
0|   |   |   | X |   |   |   |   |
 +---+---+---+---+---+---+---+---+
1|   |   |   |   |   |   |   |   |
 +---+---+---+---+---+---+---+---+
2|   |   |   |   |   |   |   |   |
 +---+---+---+---+---+---+---+---+
3|   |   |   |   |   |   |   |   |
 +---+---+---+---+---+---+---+---+
4|   |   |   |   |   |   |   |   |
 +---+---+---+---+---+---+---+---+
5|   |   |   |   |   |   |   |   |
 +---+---+---+---+---+---+---+---+
6|   |   |   |   |   |   |   |   |
 +---+---+---+---+---+---+---+---+
7|   |   |   |   |   |   |   |   |
 +---+---+---+---+---+---+---+---+
Enter the row (0-7): 7
Enter the column (0-7): 7
  0   1   2   3   4   5   6   7
 +---+---+---+---+---+---+---+---+
0|   |   |   | X |   |   |   |   |
 +---+---+---+---+---+---+---+---+
1|   |   |   |   |   |   |   |   |
 +---+---+---+---+---+---+---+---